# Store data

As calling API everytime may takes a while and we may hit the limit of API, we will store the data from API for now. 

In [1]:
import tmdbsimple as tmdb
tmdb.API_KEY = "71e259894a515060876bab2a33d6bdc9"

In [2]:
from imdb import IMDb
import pandas as pd
from PIL import Image
from StringIO import StringIO
import requests
import os
import time
from shutil import copyfile

In [3]:
dir_python_notebook = os.getcwd()
dir_movie_project = os.path.abspath(os.path.join(dir_python_notebook, os.pardir))
dir_data = os.path.join(dir_movie_project, 'data')

In [33]:
tmdb_filename = str(dir_data)+'\\drv_tmdb_movie_details.json'
tmdb_movies = pd.read_json(tmdb_filename)
tmdb_movies.head(1)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
10003,False,/lJnl8xIhplfzUBtlJLsWBwvXVBj.jpg,The Saint Collection,68000000,"[Thriller, Action, Romance, Science Fiction, A...",,10003,tt0120053,en,The Saint,...,1997-04-03,118063304,116,"[English, Pусский]",Released,Never reveal your name. Never turn your back. ...,The Saint,False,5.8,235


# IMDB

In [11]:
imdb = IMDb()
#get a movie by id
imdb_movie = imdb.get_movie('0325980')
#access attributes of the movie by dictionary keys
imdb.update(imdb_movie)
#available keys
imdb_info_column  =imdb_movie.keys()
imdb_info_column  = [str(c) for c in imdb_info_column]
imdb_info_column.sort()

In [12]:
print(len(imdb_info_column))
print(imdb_info_column)

61
['akas', 'animation department', 'art department', 'art direction', 'aspect ratio', 'assistant director', 'camera and electrical department', 'canonical title', 'cast', 'casting department', 'casting director', 'certificates', 'cinematographer', 'color info', 'costume department', 'costume designer', 'countries', 'country codes', 'cover url', 'director', 'distributors', 'editor', 'editorial department', 'full-size cover url', 'genres', 'kind', 'language codes', 'languages', 'location management', 'long imdb canonical title', 'long imdb title', 'make up', 'miscellaneous companies', 'miscellaneous crew', 'mpaa', 'music department', 'original music', 'plot', 'plot outline', 'producer', 'production companies', 'production design', 'production manager', 'rating', 'runtimes', 'set decoration', 'smart canonical title', 'smart long imdb canonical title', 'sound crew', 'sound mix', 'special effects companies', 'special effects department', 'stunt performer', 'thanks', 'title', 'top 250 rank'

In [13]:
# IMDB API shows that we can retrieve information from proprty of movies object
imdb_info_column = imdb_movie.keys_alias.keys() 
imdb_info_column.sort()
print(len(imdb_info_column))
print(imdb_info_column)

85
['actors', 'actresses', 'aka', 'also known as', 'amazon review', 'art direction by', 'casting', 'casting by', 'certificate', 'certification', 'certifications', 'cinematography', 'cinematography by', 'color', 'costume and wardrobe department', 'costume design', 'costume design by', 'country', 'cover', 'created by', 'crew members', 'crewmembers', 'directed by', 'distribution', 'distribution companies', 'distribution company', 'distributor', 'editing', 'episodes cast', 'episodes number', 'faq', 'film editing', 'film editing by', 'frequently asked questions', 'full-size cover', 'genre', 'guest', 'guest appearances', 'lang', 'language', 'make-up', 'makeup', 'makeup department', 'merchandise', 'merchandising', 'misc companies', 'misc company', 'misc crew', 'miscellaneous', 'miscellaneous company', 'miscellaneous links', 'miscellaneouscrew', 'music', 'non-original music by', 'notable tv guest appearances', 'original music by', 'other companies', 'other company', 'other crew', 'parental gui

In [20]:
for c in imdb_info_column:
    if imdb_movie.get(c) is None:
        print(c)
    #else:
    #    print (imdb_movie.get(c))

amazon review
created by
episodes cast
episodes number
faq
frequently asked questions
full-size cover
guest
guest appearances
merchandise
merchandising
miscellaneous
miscellaneous links
non-original music by
notable tv guest appearances
parental guide
photographs
sales
seasons
soundclips
special effects by
tv guests
tv schedule
videoclips


In [25]:
#for c in imdb_info_column:
#    print(c)
#    print(type(imdb_movie.get(c)))

In [34]:
# get IMDB movies that we already have data for TMDB
imdb_ids = tmdb_movies['imdb_id']
print("TMDB movie number:")
print(len(imdb_ids))
imdb_ids = [ str(imdb_id.replace('tt','')) for imdb_id in imdb_ids]

print(imdb_ids[0])

print("Valid IMDB ID number:")
print(len([x for x in imdb_ids  if x is not None and x is not ""]))

imdb_ids = [x for x in imdb_ids  if x is not None and x is not ""]


TMDB movie number:
3189
0120053
Valid IMDB ID number:
3138


In [ ]:
imdb_info_column = imdb_movie.keys_alias.keys() 
if 'imdb_id' not in imdb_info_column:
    imdb_info_column.append('imdb_id')
imdb_info_column.sort()
movies = pd.DataFrame(columns=imdb_info_column)
imdb_filename = str(dir_data)+'\\drv_imdb_movie_details.json'
imdb_filename_backup = str(dir_data)+'\\drv_imdb_movie_details_bkp.json'

count = 0
# if we already have a movie data file, we can just continue appending it instead of starting from id 1 again.
if os.path.isfile(imdb_filename):
    movies = pd.read_json(imdb_filename)
    if len(movies['imdb_id']) >0:
        movie_ids = movies['imdb_id'].tolist()
        imdb_ids = [x for x in imdb_ids if x not in movie_ids]
        imdb_info_column = movies.columns
    else:
        movies = pd.DataFrame(columns=imdb_info_column)

for i in imdb_ids[0:]:
    count += 1

    #if (count % 40 == 0):
    #    time.sleep(10) # make sure we do not hit API limit
    if (count % 500 == 0):
        movies.to_json(path_or_buf= imdb_filename)
    #skip the non-existing movie ids
    #print(i)
    try:
        imdb_movie = imdb.get_movie(i)
        imdb.update(imdb_movie)
        
        movie_details = []
        for c in imdb_info_column:
            info_field = imdb_movie.get(c)
            if info_field is not None:
                if type(info_field) is list:                    
                    if isinstance(info_field[0], ib.Person.Person):
                        info_list = []
                        for item in info_field:
                            info = {}
                            info["personID"] = item.getID()
                            info["name"] = item.get("name")
                            info_list.append(info)
                        movie_details.append(info_list)
                    elif isinstance(info_field[0], ib.Company.Company):
                        info_list = []
                        for item in info_field[c]:
                            info = {}
                            info["companyID"] = item.getID()
                            info["name"] = item.get("name")
                            info_list.append(info)
                        movie_details.append(info_list)  
                    else:
                        movie_details.append(info_field)
                else:
                    movie_details.append(info_field)
            else:
                if c == "imdb_id":
                    movie_details.append(i)
                else:
                    movie_details.append(None)
                
        movies.loc[len(movies.index)] = movie_details
        
    except Exception:
        #print(i)
        continue
        
movies.to_json(path_or_buf= imdb_filename)


2017-04-04 17:23:31,178 WARNING [imdbpy.parser.http.urlopener] C:\Users\Chrystal\Anaconda2\envs\py27\lib\site-packages\imdb\parser\http\__init__.py:248: 404 code returned for //akas.imdb.com/title/tt1597526/combined: Not Found (headers: Date: Tue, 04 Apr 2017 21:23:28 GMT
Server: HTTPDaemon
X-Frame-Options: SAMEORIGIN
Cache-Control: private
Content-Type: text/html; charset=iso-8859-1
Set-Cookie: cache=BCYpL_1IAXV6UiaaSrEnNKK_iEa4-MyMeYS8uec1GHVDQwBN5VuhTOfA9Ukf2EnbQ0h2Ajam6-b9Rl-rfUwIdMIUMLPS6suQAht9GikxjXOSXY8;path=/;domain=.imdb.com
Set-Cookie: uu=BCYhFCe2KKOcCP_l3CPFyRvuKUe4-MyMeYS8uec1GHVDQwAXoU1ppsC9HE7xYwB-Z-SQPWZnsLIAJoOpSTTa2CgC5tkCBAtgofkn6w23QVp1OnZGwimhkhN6hN2w3tZYVDXOzEZhhrn7S9I-XRcCncHjg4NGiKvY-x8bkav33kn1cz1MgFWEDXiKMWfQTj4BK8Go96vsb4nWJBfrUqbZ2e8pLPqDVYEg2x2seARG5-V3rTHF4BMYvKxBiet8UBCxODkuUivC3Kk5VwrOGlDLHdzq0UB0dMUcyVr3YliSlSKEjjk6At_S5FDD66zzSHC5zVFU4hYvylnuPSTkfgM21p577g;expires=Thu, 30 Dec 2037 00:00:00 GMT;path=/;domain=.imdb.com
Set-Cookie: cache=BCYkx9DB_sJHXtU1R

In [ ]:
#make a backup in case of corruption
copyfile(imdb_filename, imdb_filename_backup) 

In [7]:
imdb_filename = str(dir_data)+'\\drv_imdb_movie_details.json'
imdb_movies = pd.read_json(imdb_filename)
imdb_movies.head(5)

,actors,actresses,aka,also known as,amazon review,art direction by,casting,casting by,certificate,certification,...,soundclips,special effects by,special effects company,stunts,tv guests,tv schedule,user rating,videoclips,visual effects by,writing credits
0,"[{u'personID': u'0000552', u'name': u'Eddie Mu...","[{u'personID': u'0000552', u'name': u'Eddie Mu...","[Un detective suelto en Hollywood::Argentina, ...","[Un detective suelto en Hollywood::Argentina, ...",NaN,"[{u'personID': u'0613468', u'name': u'James J....","[{u'personID': u'0799557', u'name': u'Margery ...","[{u'personID': u'0799557', u'name': u'Margery ...","[Argentina:16, Australia:M, Brazil:12, Canada:...","[Argentina:16, Australia:M, Brazil:12, Canada:...",...,NaN,NaN,None,"[{u'personID': u'0010842', u'name': u'Christop...",NaN,NaN,7.3,NaN,"[{u'personID': u'0344477', u'name': u'Steve Gr...","[{u'personID': u'0677943', u'name': u'Daniel P..."
1,"[{u'personID': u'0000160', u'name': u'Ethan Ha...","[{u'personID': u'0000160', u'name': u'Ethan Ha...","[Antes del atardecer::Argentina, International...","[Antes del atardecer::Argentina, International...",NaN,None,None,None,"[Argentina:Atp, Australia:M, Austria:0, Brazil...","[Argentina:Atp, Australia:M, Austria:0, Brazil...",...,NaN,NaN,"[{u'name': u'Custom Film Effects', u'companyID...",None,NaN,NaN,8.0,NaN,"[{u'personID': u'1980594', u'name': u'Ryan Bea...","[{u'personID': u'0000500', u'name': u'Richard ..."
10,"[{u'personID': u'0000136', u'name': u'Johnny D...","[{u'personID': u'0000136', u'name': u'Johnny D...","[P.O.T.C. 2::USA (promotional abbreviation), P...","[P.O.T.C. 2::USA (promotional abbreviation), P...",NaN,"[{u'personID': u'0188692', u'name': u'Bruce Cr...","[{u'personID': u'0150522', u'name': u'Denise C...","[{u'personID': u'0150522', u'name': u'Denise C...","[Argentina:13, Australia:M, Brazil:12, Canada:...","[Argentina:13, Australia:M, Brazil:12, Canada:...",...,NaN,NaN,"[{u'name': u'Creative Makeup', u'companyID': u...","[{u'personID': u'0022315', u'name': u'Robert A...",NaN,NaN,7.3,NaN,"[{u'personID': u'0008671', u'name': u'Mimi Abe...","[{u'personID': u'0254645', u'name': u'Ted Elli..."
11,"[{u'personID': u'0000235', u'name': u'Uma Thur...","[{u'personID': u'0000235', u'name': u'Uma Thur...","[Kill Bill::USA (informal short title), Kill B...","[Kill Bill::USA (informal short title), Kill B...",NaN,"[{u'personID': u'0103011', u'name': u'Daniel B...","[{u'personID': u'0535338', u'name': u'Koko Mae...","[{u'personID': u'0535338', u'name': u'Koko Mae...","[Argentina:16, Australia:R18+, Brazil:18, Cana...","[Argentina:16, Australia:R18+, Brazil:18, Cana...",...,NaN,NaN,"[{u'name': u'KNB EFX Group', u'companyID': u'0...","[{u'personID': u'0011121', u'name': u'Keith Ad...",NaN,NaN,8.1,NaN,"[{u'personID': u'0105802', u'name': u'Kevin Br...","[{u'personID': u'0000233', u'name': u'Quentin ..."
12,"[{u'personID': u'0671231', u'name': u'Matti Pe...","[{u'personID': u'0671231', u'name': u'Matti Pe...",[Shadows in Paradise::International (English t...,[Shadows in Paradise::International (English t...,NaN,"[{u'personID': u'0383991', u'name': u'Pertti H...",None,None,"[Finland:S, Iceland:L, Singapore:PG, UK:12::(v...","[Finland:S, Iceland:L, Singapore:PG, UK:12::(v...",...,NaN,NaN,None,None,NaN,NaN,7.6,NaN,None,"[{u'personID': u'0442454', u'name': u'Aki Kaur..."
